# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.chat_models import QianfanChatEndpoint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

print(f"QIANFAN_AK={os.environ['QIANFAN_AK']}")
print(f"QIANFAN_SK={os.environ['QIANFAN_SK']}")

import warnings
warnings.filterwarnings("ignore")

QIANFAN_AK=HCCPsQy5p0Ex1rSEL6oorGQb
QIANFAN_SK=TCVRfPCfbtLr0eDPZRXcywMxcgaNuLDE


Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [7]:
# # account for deprecation of LLM model
# import datetime
# # Get the current date
# current_date = datetime.datetime.now().date()

# # Define the date after which the model should be set to "gpt-3.5-turbo"
# target_date = datetime.date(2024, 6, 12)

# # Set the model variable based on the current date
# if current_date > target_date:
#     llm_model = "gpt-3.5-turbo"
# else:
#     llm_model = "gpt-3.5-turbo-0301"

In [8]:
#!pip install pandas

In [9]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [10]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9) 

In [13]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [14]:
chain = LLMChain(llm=llm, prompt=prompt)

In [15]:
product = "Queen Size Sheet Set"
chain.run(product)

[INFO] [05-09 11:49:38] oauth.py:222 [t:32800]: trying to refresh access_token for ak `HCCPsQ***`
[INFO] [05-09 11:49:40] oauth.py:237 [t:32800]: sucessfully refresh access_token


"Choosing a name for a company that manufactures Queen Size Sheet Sets depends on various factors, including the brand's image, target audience, and market positioning. Here are some suggestions that might help:\n\n1. **Luxury Linens Co.**: This name suggests a high-end, luxurious product, which could be appealing to customers looking for quality sheets.\n2. **Queen's Comfort Bedding**: Directly references the product (Queen Size Sheets) while also suggesting comfort and royalty, which are desirable qualities in bedding.\n3. **Elegant Sleep Solutions**: This name implies that the sheets are not just a product but a solution for a better sleep experience, emphasizing elegance.\n4. **Boutique Bedding Co.**: Suggests a more personalized or unique approach to bedding, which could attract customers looking for something different.\n5. **Royal Rest Sheets**: Combines the idea of royalty (fitting for Queen Size) with restful sleep, creating a positive association.\n\nRemember, the best name f

## SimpleSequentialChain

In [16]:
from langchain.chains import SimpleSequentialChain

In [17]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9) 

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [18]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [19]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [20]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Choosing a name for a company that manufactures Queen Size Sheet Sets depends on the branding, target audience, and overall marketing strategy of the business. However, here are some suggestions that might help:

1. **Luxury Linen Crafters**: This name suggests high-quality, handmade sheets that are luxurious and crafted with care.
2. **Queen's Comfort Collection**: This name directly references the product (Queen Size Sheet Sets) and suggests comfort and elegance.
3. **Boutique Bed Sheets**: This name suggests a more boutique, upscale approach to bed sheets, appealing to customers who appreciate unique and stylish home decor.
4. **Palatial Sheets**: This name evokes a sense of grandeur and luxury, appealing to customers who want to feel like royalty in their own beds.
5. **Elegant Ease Bedding**: This name combines elements of elegance and comfort, appealing to customers who value both style and practicality.

Remember, the best name for

'Company manufactures luxurious, high-quality Queen Size Sheet Sets, prioritizing comfort and elegance.'

## SequentialChain

In [21]:
from langchain.chains import SequentialChain

In [23]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9) 

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [24]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [25]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [26]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [27]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [28]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '抱歉，这个功能暂未开放上线。您也可以用中文或英文问我一些其他的问题，我会尽力为您解答。',
 'summary': '该评论指出某项功能尚未开放上线，但用户仍可以询问中英文的其他问题，评论者会尽力回答。',
 'followup_message': '非常感谢您的理解和耐心。针对您提到的功能，我们深表歉意，因为目前该功能确实还未开放上线。我们正在努力完善并尽快推出，以满足您的需求。\n\n在此期间，如果您有任何中英文的其他问题，欢迎随时向我们提问。我们将竭尽所能为您提供满意的答案和解决方案。再次感谢您的支持和关注，期待能为您带来更好的服务体验。'}

## Router Chain

In [29]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [30]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [31]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [32]:
# llm = ChatOpenAI(temperature=0, model=llm_model)
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9) 

In [33]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [34]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [35]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [36]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [37]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [38]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by an idealized physical body known as a black body. A black body is an object that absorbs all incident radiation, regardless of frequency or angle of incidence, and re-emits it as thermal radiation. The radiation emitted by a black body is characterized by a specific spectrum and intensity that depends only on the body's temperature. This radiation is called black body radiation.\n\nThe spectrum of black body radiation is described by Planck's law, which states that the intensity of radiation emitted at a given frequency is proportional to the temperature of the black body and varies with frequency in a specific way. The shape of the spectrum changes with temperature, shifting to higher frequencies as the temperature increases.\n\nBlack body radiation plays a fundamental role in many areas of physics, including thermodynamics, quantum mechanics, and astronomy. For example, the radiation emitted by stars can be appr

In [39]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to the question "what is 2 + 2" is 4. This is a basic addition problem in mathematics, where two numbers are added together to produce a sum. In this case, 2 + 2 equals 4.'

In [40]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"DNA, or deoxyribonucleic acid, is the genetic material of all living organisms, including humans. It is the carrier of genetic information and is responsible for storing and transmitting the genetic code of the organism. Every cell in the human body contains DNA because it is necessary for the cell to function properly.\n\nFirstly, DNA provides the instructions for the production of proteins, which are essential for the structure, function, and regulation of the human body's cells, tissues, and organs. Proteins play a crucial role in almost all biological processes, including metabolism, cell division, and immune responses.\n\nSecondly, DNA also contains the genetic information that determines the unique characteristics of each individual, such as eye color, hair color, and blood type. This information is passed down from generation to generation through DNA.\n\nFinally, DNA is also involved in the repair and maintenance of the cell's genetic material, ensuring the stability and integ

Reminder: Download your notebook to you local computer to save your work.